In [33]:
import pandas as pd

df = pd.read_csv("data/nasdaq_screener_1743382889678.csv")

df.dtypes

Symbol         object
Name           object
Last Sale      object
Net Change    float64
% Change       object
Market Cap    float64
Country        object
IPO Year      float64
Volume          int64
Sector         object
Industry       object
dtype: object

In [ ]:
from sqlalchemy import create_engine, Table, Column, String, Date, BigInteger, MetaData
import os
from datetime import datetime

username = os.environ.get("AZURE_SQL_USER")
password = os.environ.get("AZURE_SQL_PASSWORD")
server   = os.environ.get("AZURE_SQL_SERVER")
database = os.environ.get("AZURE_SQL_DB")

def clean_df(df):
    df = df.where(pd.notnull(df), 0)
    return df

metadata = MetaData()
all_listed_stocks = Table("all_listed_stocks",metadata,
                        Column("Symbol", String(50), primary_key=True, nullable=False),
                        Column("Name", String(100), primary_key=False, nullable=True),
                        Column("Country", String(50),primary_key=True, nullable=False),
                        Column("Industry", String(100), nullable=True),
                        Column("Sector", String(100), nullable=True),
                        Column("created_date", Date, nullable=True),
                        Column("last_update_date", Date, nullable=True),
                        Column("create_id", String(100), nullable=True)
                        )
columns_to_insert = ["Symbol","Name","Country","Industry","Sector"]
df = pd.read_csv("data/nasdaq_screener_1743382889678.csv",usecols= columns_to_insert,nrows=2)
df["created_date"] = datetime.today()
df["last_update_date"] = datetime.today()
df["create_id"] = "ONE_TIME_LOAD"

df = clean_df(df)
total_records = len(df)
records = df.to_dict("records")
print(records)

connection_string = f"mssql+pymssql://{username}:{password}@{server}/{database}?login_timeout=120&timeout=300"
engine = create_engine(connection_string)

with engine.begin() as connection:
    for i in range(0,total_records,1000):
        chunk = records[i:i+1000]
        result = connection.execute(all_listed_stocks.insert().values(chunk))
        print(f"number of rows inserted {result.rowcount}")




[{'Symbol': 'AACBU', 'Name': 'Artius II Acquisition Inc. Units', 'Country': 'United States', 'Sector': 'Finance', 'Industry': 'Blank Checks', 'created_date': Timestamp('2025-04-07 20:20:19.611552'), 'last_update_date': Timestamp('2025-04-07 20:20:19.611552'), 'create_id': 'ONE_TIME_LOAD'}, {'Symbol': 'AAL', 'Name': 'American Airlines Group Inc. Common Stock', 'Country': 'United States', 'Sector': 'Consumer Discretionary', 'Industry': 'Air Freight/Delivery Services', 'created_date': Timestamp('2025-04-07 20:20:19.611552'), 'last_update_date': Timestamp('2025-04-07 20:20:19.611552'), 'create_id': 'ONE_TIME_LOAD'}]
number of rows inserted 2


-s a2663430-8c4a-45ec-a062-881eb856f247 -g stock-dashboard -a StockDataCollector -m pyodbc -v 5.2.0

In [35]:
df

,Symbol,Name,Country,Sector,Industry,created_date,last_update_date,create_id
0,AACBU,Artius II Acquisition Inc. Units,United States,Finance,Blank Checks,2025-04-07 20:20:19.611552,2025-04-07 20:20:19.611552,ONE_TIME_LOAD
1,AAL,American Airlines Group Inc. Common Stock,United States,Consumer Discretionary,Air Freight/Delivery Services,2025-04-07 20:20:19.611552,2025-04-07 20:20:19.611552,ONE_TIME_LOAD


In [36]:
df1=df[["Symbol","Name","Sector","Industry"]]

records = df1.to_dict("records")



In [37]:
records

[{'Symbol': 'AACBU',
  'Name': 'Artius II Acquisition Inc. Units',
  'Sector': 'Finance',
  'Industry': 'Blank Checks'},
 {'Symbol': 'AAL',
  'Name': 'American Airlines Group Inc. Common Stock',
  'Sector': 'Consumer Discretionary',
  'Industry': 'Air Freight/Delivery Services'}]

In [38]:
df.head(5)

,Symbol,Name,Country,Sector,Industry,created_date,last_update_date,create_id
0,AACBU,Artius II Acquisition Inc. Units,United States,Finance,Blank Checks,2025-04-07 20:20:19.611552,2025-04-07 20:20:19.611552,ONE_TIME_LOAD
1,AAL,American Airlines Group Inc. Common Stock,United States,Consumer Discretionary,Air Freight/Delivery Services,2025-04-07 20:20:19.611552,2025-04-07 20:20:19.611552,ONE_TIME_LOAD
